## Import library 

In [143]:
import sys
sys.path.insert(1, '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib')
import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math 
import timeit

from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm

**Simulated data with true effect is 100**

In [144]:
true_effect = 30
data = dowhy.datasets.linear_dataset(beta=true_effect,
        num_common_causes=5,
        num_instruments = 0,
        num_samples=10000,
        treatment_is_binary=True)
df = data["df"].copy()
df['v0'] = [1 if x == True else 0 for x in df['v0']]

In [88]:
df.head()

,W0,W1,W2,W3,W4,v0,y
0,0.017768,-0.495496,-1.544749,-0.734019,1.767539,1,23.679874
1,-0.776575,0.119851,1.256166,0.617782,1.533567,1,45.317776
2,-0.421549,-0.246840,0.796873,2.153733,0.265466,1,52.790256
3,1.654084,1.966534,2.536988,-1.139203,0.713396,1,51.772308
4,-0.440972,0.553262,1.211194,0.453515,0.002074,1,40.581794


## Summary
$$p_1 = E[A = 1 | X]$$
$$p_0 = E[A = 0 | X]$$

$$q_1 = \frac{\delta p_0}{\delta p_0 + p_1}$$
$$\Psi (\delta ) = q1*\mathbb{E}[Y | X,A=1] + q_0*\mathbb{E}[Y | X,A=0]$$

**Influence function**

$$\Psi (\delta ) = q_1*\phi_1 + q_0*\phi_0 + \frac{\delta*(\mu(X,1) - \mu(X,0))}{(\delta*p_1 + p_0)^2}$$ 
$$\phi_a = \frac{\mathbb{1}(A=a)}{\pi(a|x)}(Y - \mu(X,A)) +  \mu(X,a)$$

## Propensity Score 

In [89]:
logreg = LogisticRegression()
logreg.fit(df[data['common_causes_names']], df[data['treatment_name'][0]])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [90]:
y_pred = logreg.predict(df[data['common_causes_names']])
print("Accuracy:",metrics.accuracy_score(df[data['treatment_name'][0]], y_pred))
print("Precision:",metrics.precision_score(df[data['treatment_name'][0]], y_pred))
print("Recall:",metrics.recall_score(df[data['treatment_name'][0]], y_pred))

Accuracy: 0.9798
Precision: 0.9866920152091255
Recall: 0.9892780557541101


In [91]:
rfc = RandomForestClassifier(max_depth=50, random_state=0)
rfc.fit(df[data['common_causes_names']], df[data['treatment_name'][0]])

y_pred = rfc.predict(df[data['common_causes_names']])
print("Accuracy:",metrics.accuracy_score(df[data['treatment_name'][0]], y_pred))
print("Precision:",metrics.precision_score(df[data['treatment_name'][0]], y_pred))
print("Recall:",metrics.recall_score(df[data['treatment_name'][0]], y_pred))

Accuracy: 1.0
Precision: 1.0
Recall: 1.0


In [92]:
df['ps_1'] = logreg.predict_proba(df[data['common_causes_names']])[:,1]
df['ps_0'] = logreg.predict_proba(df[data['common_causes_names']])[:,0]


## K-nearest neighbor

In [93]:
att, atc, est= pse.propensity_score_estimator_k_nearest_neighbor(df, data['treatment_name'][0], data['outcome_name'], 'ps_1')

In [94]:
print("Average effect on treated: {}".format(att))
print("Average effect on control: {}".format(atc))
print("Average effect: {}".format(est))
print("MAE with ground truth {}".format(abs(est - true_effect)))

Average effect on treated: 45.19316322398087
Average effect on control: 36.57656965680168
Average effect: 40.50870439654845
MAE with ground truth 10.508704396548453


## Incremental score 

**Incremental propensity score:**
$$q_1 = \frac{\delta p_0}{\delta p_0 + p_1}$$


In [122]:
delta = 1.5
df['incre_ps_1'] = (delta * df['ps_1']) / (delta * df['ps_1'] + df['ps_0'])
df['incre_ps_0'] = 1 - df['incre_ps_1']

In [123]:
df.head()

,W0,W1,W2,W3,W4,v0,y,ps_1,ps_0,incre_ps_1,incre_ps_0,predicted_y,treated_cf_outcome,control_cf_outcome,incre_effect
0,0.017768,-0.495496,-1.544749,-0.734019,1.767539,1,23.679874,0.85,0.15,0.894737,0.105263,24.718352,-8.884475,24.718352,-5.347336
1,-0.776575,0.119851,1.256166,0.617782,1.533567,1,45.317776,1.00,0.00,1.000000,0.000000,45.660158,-1.243305,45.660158,-1.243305
2,-0.421549,-0.246840,0.796873,2.153733,0.265466,1,52.790256,1.00,0.00,1.000000,0.000000,52.631210,7.411488,52.631210,7.411488
3,1.654084,1.966534,2.536988,-1.139203,0.713396,1,51.772308,1.00,0.00,1.000000,0.000000,51.229261,-4.118942,51.229261,-4.118942
4,-0.440972,0.553262,1.211194,0.453515,0.002074,1,40.581794,0.99,0.01,0.993311,0.006689,40.256289,0.870206,40.256289,1.133658


**Regress Y on X and A:**

$$\mu = \mathbb{E}[Y | X,A]$$

In [124]:
features = data['common_causes_names'] + data['treatment_name']
rf = RandomForestRegressor(n_estimators = 100, random_state = 1)
rf.fit(df[features], df['y'])

# rf = GradientBoostingRegressor(random_state=1)
# rf.fit(df[features], df['y'])


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1, verbose=0, warm_start=False)

In [125]:
df['predicted_y'] = rf.predict(df[features])

In [126]:
explained_variance_score(df[data['outcome_name'][0]],df['predicted_y'])

**Estimator:**
$$\Psi (\delta ) = q1*\mathbb{E}[Y | X,A=1] + q_0*\mathbb{E}[Y | X,A=0]$$


In [134]:
## Compute counterfactual outcome with no treatment
df_pos = df.copy()
df_pos['v0'] = 1
df['treated_cf_outcome'] = rf.predict(df_pos[features])

## Compute counterfactual outcome with treatment
df_control = df.copy()
df_control['v0'] = 0
df['control_cf_outcome'] = rf.predict(df_control[features])


In [136]:
df['incre_effect'] = df['incre_ps_1']*df['treated_cf_outcome'] + df['incre_ps_0']*df['control_cf_outcome']

In [137]:
df['y']

0       23.679874
1       45.317776
2       52.790256
3       51.772308
4       40.581794
          ...    
9995    34.344335
9996   -21.458896
9997    50.978966
9998    83.977471
9999    37.678802
Name: y, Length: 10000, dtype: float64

In [138]:
rf.predict(df[features])

array([24.71835163, 45.6601577 , 52.63121   , ..., 51.11629889,
       81.51583067, 37.4700542 ])

In [139]:
df.head()

,W0,W1,W2,W3,W4,v0,y,ps_1,ps_0,incre_ps_1,incre_ps_0,predicted_y,treated_cf_outcome,control_cf_outcome,incre_effect
0,0.017768,-0.495496,-1.544749,-0.734019,1.767539,1,23.679874,0.85,0.15,0.894737,0.105263,24.718352,24.718352,-8.884475,21.181212
1,-0.776575,0.119851,1.256166,0.617782,1.533567,1,45.317776,1.00,0.00,1.000000,0.000000,45.660158,45.660158,-1.243305,45.660158
2,-0.421549,-0.246840,0.796873,2.153733,0.265466,1,52.790256,1.00,0.00,1.000000,0.000000,52.631210,52.631210,7.411488,52.631210
3,1.654084,1.966534,2.536988,-1.139203,0.713396,1,51.772308,1.00,0.00,1.000000,0.000000,51.229261,51.229261,-4.118942,51.229261
4,-0.440972,0.553262,1.211194,0.453515,0.002074,1,40.581794,0.99,0.01,0.993311,0.006689,40.256289,40.256289,0.870206,39.992837


In [140]:
np.mean(df['incre_effect'])

In [141]:
print("MAE with ground truth {}".format(abs(np.mean(df['incre_effect']) - true_effect)))


MAE with ground truth 8.599980779487083


## Incremental score with library

In [106]:
df_lib = data['df']
ipse.incremenal_ps_score_estimator(df,
                                   data['treatment_name'], 
                                   data['common_causes_names'], 
                                   rfc, 
                                   rf, 
                                   delta)

## Influence function 

**Influence function** $\mathbb{1}$

$$\Psi (\delta ) = q_0*\phi_0 + q_1*\phi_1 + \frac{\delta*(\mu(X,1) - \mu(X,0)*(A - p_1)}{(\delta*p_1 + p_0)^2}$$ 

$$\phi_a = \frac{\mathbb{1}(A=a)}{\pi(a|x)}(Y - \mu(X,A)) +  \mu(X,a)$$

In [107]:
# Y - mu(X,A)
residual = df[data['outcome_name'][0]] - df['predicted_y']

w0 = [0 if df.loc[i, 'v0'] == 1 else 1/df.loc[i, 'ps_0'] for i in range(len(df))]
w1 = [0 if df.loc[i, 'v0'] == 0 else 1/df.loc[i, 'ps_1'] for i in range(len(df))]
w0 = np.array(w0)
w1 = np.array(w1)

phi_0 = w0 * residual + df['treated_cf_outcome'] 
phi_1 = w1 * residual + df['control_cf_outcome']

m0 = df['incre_ps_1']*phi_1 + df['incre_ps_0']*phi_0

m1 = (delta*(df['treated_cf_outcome'] - df['control_cf_outcome'])
     *(df['v0'] - df['ps_1']))/(delta*df['ps_1'] + df['ps_0'])**2


In [108]:
influence = m0 + m1 - df['incre_effect']

In [109]:
influence

0       21.712108
1       46.561081
2       45.378769
3       55.891250
4       38.921440
          ...    
9995    34.543622
9996   -34.572121
9997    47.371761
9998    77.108886
9999    39.525828
Length: 10000, dtype: float64

## Inverse Probability Weighted Estimator

## Proposed Estimation

In [110]:
features = data['common_causes_names'] + data['treatment_name']


In [111]:
df_expr = data["df"].copy()
df_expr['v0'] = [1 if x == True else 0 for x in df_expr['v0']]

In [112]:
df_expr.head()

,W0,W1,W2,W3,W4,v0,y
0,0.017768,-0.495496,-1.544749,-0.734019,1.767539,1,23.679874
1,-0.776575,0.119851,1.256166,0.617782,1.533567,1,45.317776
2,-0.421549,-0.246840,0.796873,2.153733,0.265466,1,52.790256
3,1.654084,1.966534,2.536988,-1.139203,0.713396,1,51.772308
4,-0.440972,0.553262,1.211194,0.453515,0.002074,1,40.581794


Split data into $D_0$ and $D_1$

In [75]:
D0, D1 = train_test_split(df_expr, train_size = 0.9, random_state = 1)

In [30]:
D0 = D0.reset_index().drop(columns = ['index'])
D1 = D1.reset_index().drop(columns = ['index'])

### Fit treatment model
Regress $A_t \sim H_t$ in $D_0$, obtain preds $\pi_t(H_t)$ in D

In [31]:
logreg = LogisticRegression()
logreg.fit(D0[data['common_causes_names']], D0['v0'])

df_expr['ps_1'] = logreg.predict_proba(df_expr[data['common_causes_names']])[:,1]
df_expr['ps_0'] = 1 - df_expr['ps_1']

### Compute weights
Compute weight on on $D_1$
$$W_t = \frac{delta*A_1 + A_0}{\hat{p_1} + \hat{p_0}}$$ 

Compute in $D_1$
$$V_t = \frac{p_1*\hat{p_0} + \delta*p_0*\hat{p_1}}{\frac{\delta}{1-\delta}} $$



In [32]:
D1['ps_1'] = logreg.predict_proba(D1[data['common_causes_names']])[:,1]
D1['ps_0'] = 1 - D1['ps_1']

D1['incre_ps_1'] = (delta * D1['ps_1']) / (delta * D1['ps_1'] + D1['ps_0'])
D1['incre_ps_0'] = 1 - D1['incre_ps_1']


In [33]:
delta = 0.5
w_t = (delta*D1['v0'] + 1 - D1['v0']) / (delta*D1['ps_1'] + D1['ps_0'])
v_t = (D1['v0']*(D1['ps_0']) 
                - (1 - D1['v0'])*D1['ps_1']*delta) / (delta / (1-delta))

### Fit outcome models
Regress $R_{t+1} \sim (H_t , A_t)$ in $D_0$, obtain preds $m_t(H_t,a)$ in D

In [34]:
# whole dat, random > gradient
# D0, D1 => random on D1 < gradient on D1

rf = GradientBoostingRegressor(random_state=0, n_estimators = 1000)
rf.fit(D0[features], D0[data['outcome_name']])

y_pred = rf.predict(D1[features])
y_true = D1[data['outcome_name'][0]]
explained_variance_score(y_true, y_pred)

In [35]:
D1.head(100)

,W0,W1,W2,W3,W4,v0,y,ps_1,ps_0,incre_ps_1,incre_ps_0
0,0.534170,-0.844949,0.646221,-0.374063,-0.039976,0,0.136776,0.130164,0.869836,0.183316,0.816684
1,1.029547,-1.670569,-1.252912,-0.765659,1.686114,1,13.619552,0.454465,0.545535,0.555476,0.444524
2,0.056188,-1.080396,0.093991,-1.292528,3.615162,1,20.210184,0.973937,0.026063,0.982473,0.017527
3,0.715787,-2.612068,-0.126430,-0.149371,0.330459,0,-1.399294,0.026875,0.973125,0.039778,0.960222
4,1.100687,0.258205,0.820043,1.077088,0.600246,1,24.424315,0.991218,0.008782,0.994128,0.005872
...,...,...,...,...,...,...,...,...,...,...,...
95,1.344203,-1.401189,-0.122839,-0.010091,0.074416,0,3.180488,0.193378,0.806622,0.264494,0.735506
96,-1.850544,-0.599368,0.039023,-1.077219,-0.476228,0,-16.151253,0.003933,0.996067,0.005888,0.994112
97,0.485685,-1.386320,0.726348,-0.477247,1.361227,1,15.303535,0.523060,0.476940,0.621935,0.378065
98,0.877588,-0.325708,-0.028954,-1.211755,1.963579,1,17.500273,0.867856,0.132144,0.907845,0.092155


In [36]:
## Compute counterfactual outcome with no treatment
df_pos = D1.copy()
df_pos['v0'] = 1
D1['treated_cf_outcome'] = rf.predict(df_pos[features])
m1 = np.mean(D1['treated_cf_outcome'])

## Compute counterfactual outcome with treatment
df_neg = D1.copy()
df_neg['v0'] = 0
D1['control_cf_outcome'] = rf.predict(df_neg[features])
m0 = np.mean(D1['control_cf_outcome'])

In [37]:
D1['predicted_y'] = rf.predict(D1[features])

In [38]:
r_t = (delta*m1*D1['incre_ps_1'] + m0*D1['incre_ps_0']) / (delta*D1['incre_ps_1'] + D1['incre_ps_0'])

$$ \varphi = \tilde{W_t}*Y + \sum{\tilde{W_t}*V_t*R_t}  $$

In [52]:
effect = w_t*D1[data['outcome_name']] + w_t*v_t*r_t

In [53]:
sum(w_t*v_t*r_t)

### Compute estimator

In [40]:
mu_effect = np.mean(effect)

In [41]:
mu_effect

In [42]:
print("MAE with groundtruth {}".format(true_effect - mu_effect))

MAE with groundtruth 3.615815797019164


### Compute with library

In [76]:
df_expr = data["df"].copy()
df_expr['v0'] = [1 if x == True else 0 for x in df_expr['v0']]

D0, D1 = train_test_split(df_expr, train_size = 0.9, random_state = 1)
D0 = D0.reset_index().drop(columns = ['index'])
D1 = D1.reset_index().drop(columns = ['index'])

In [77]:
def sample_estimator(D0, D1, treatment, covariate, outcome, delta):
    features = covariate + treatment
    
    ## fit treatment effect on D0
    logreg = LogisticRegression()
    logreg.fit(D0[covariate], D0[treatment[0]])

    ## predict on D1
    D1['ps_1'] = logreg.predict_proba(D1[covariate])[:,1]
    D1['ps_0'] = 1 - D1['ps_1']
    
    D1['incre_ps_1'] = (delta * D1['ps_1']) / (delta * D1['ps_1'] + D1['ps_0'])
    D1['incre_ps_0'] = 1 - D1['incre_ps_1'] 
    ## compute weight
    w_t = (delta*D1[treatment[0]] + 1 - D1[treatment[0]]) / (delta*D1['ps_1'] + D1['ps_0'])
    v_t = (1-delta)*(D1[treatment[0]]*D1['ps_0']- (1-D1[treatment[0]])*D1['ps_1']*delta) / delta
    
    ## fit outcome on D0
    gbr = GradientBoostingRegressor(random_state=0, n_estimators = 1000)
    gbr.fit(D0[features], D0[outcome])

    ## Compute counterfactual outcome on D1
    df_pos = D1.copy()
    df_pos[treatment[0]] = 1
    D1['treated_cf_outcome'] = gbr.predict(df_pos[features])

    df_neg = D1.copy()
    df_neg[treatment[0]] = 0
    D1['control_cf_outcome'] = gbr.predict(df_neg[features])

    m0 = np.mean(D1['treated_cf_outcome'])
    m1 = np.mean(D1['control_cf_outcome'])



    
    r_t = (delta*m1*D1['incre_ps_1'] + m0*D1['incre_ps_0']) / (delta*D1['incre_ps_1'] + D1['incre_ps_0'])

    effect = w_t*D1[outcome] + w_t*v_t*r_t
    
    del logreg
    del gbr
    del D0, D1
    del df_pos, df_neg
    
    return np.mean(effect)


In [78]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 1)
effect = []
fold = 0
delta = np.linspace(0.5, 5, 5)
for sample_delta in tqdm(delta):
    
    D0, D1 = train_test_split(df_expr, train_size = 0.9, random_state = 1)
    
    D0 = D0.reset_index().drop(columns = ['index'])
    D1 = D1.reset_index().drop(columns = ['index'])
    print("Delta {}".format(sample_delta))
    e = sample_estimator(D0, 
                        D1, 
                        data['treatment_name'], 
                        data['common_causes_names'], 
                        data['outcome_name'], 
                        sample_delta)
    effect.append(e)
    
    

Delta 0.5
Delta 1.625
Delta 2.75
Delta 3.875
Delta 5.0


In [79]:
effect

In [83]:
estimate_efect = abs(np.mean(effect))

In [84]:
print("MAE with groundtruth {}".format(true_effect - estimate_efect))

MAE with groundtruth 5.789081318291622


## Z-Estimator Algorithm

In [327]:
true_effect = 100
data = dowhy.datasets.linear_dataset(beta=true_effect,
        num_common_causes=5,
        num_instruments = 0,
        num_samples=20000,
        treatment_is_binary=True)
df = data["df"].copy()
df['v0'] = [1 if x == True else 0 for x in df['v0']]

In [274]:
delta = 5


In [275]:
features = data['common_causes_names'] + data['treatment_name']

In [276]:
logreg = LogisticRegression()
logreg.fit(df[data['common_causes_names']], df['v0'])


## Compute propensity score
df['ps_1'] = logreg.predict_proba(df[data['common_causes_names']])[:,1]
df['ps_0'] = 1 - df['ps_1']

In [277]:
df['incre_ps_1'] = (delta * df['ps_1']) / (delta * df['ps_1'] + df['ps_0'])
df['incre_ps_0'] = 1 - df['incre_ps_1']

In [278]:
w_t = (delta*df['v0'] + 1 - df['v0']) / (delta*df['ps_1'] + df['ps_0'])
v_t = (df['v0']*(df['ps_0']) 
                - (1 - df['v0'])*df['ps_1']*delta) / (delta / (1-delta))

In [279]:
rf = GradientBoostingRegressor(random_state=0, n_estimators = 1000)
rf.fit(df[features], df[data['outcome_name']])

y_pred = rf.predict(df[features])
y_true = df[data['outcome_name'][0]]
explained_variance_score(y_true, y_pred)

In [280]:
## Compute counterfactual outcome with no treatment
df_pos = df.copy()
df_pos['v0'] = 1
df['treated_cf_outcome'] = rf.predict(df_pos[features])

## Compute counterfactual outcome with treatment
df_neg = df.copy()
df_neg['v0'] = 0
df['control_cf_outcome'] = rf.predict(df_neg[features])

In [281]:
r_t = (df['treated_cf_outcome']*df['incre_ps_1'] + df['control_cf_outcome']*df['incre_ps_0'])

In [322]:
df.head(10)

,W0,W1,W2,W3,W4,v0,y
0,0.096206,0.971196,1.203145,0.015968,-0.077362,1,176.727183
1,-0.020612,0.061670,0.969788,-1.632420,0.610458,0,-15.569232
2,-0.614938,-1.557552,-0.139381,0.426463,-0.056255,0,-53.401513
3,-0.829188,-0.672672,-0.623049,1.072348,-1.648874,0,-75.760870
4,-0.189506,0.701842,0.958815,0.925255,0.760471,1,232.048812
5,-1.103408,0.489900,-0.415640,-0.352104,0.228377,0,-17.213169
6,-0.778890,-1.343628,-1.040610,0.341746,1.274583,0,-27.209319
7,-0.777230,-1.316997,2.042424,-0.521080,0.143427,0,-7.182581
8,-0.011835,-1.710567,0.911075,-0.370184,0.955195,0,-10.309262
9,-0.553186,0.108744,0.267395,0.371012,2.150052,1,213.127386


In [283]:
np.mean(df['treated_cf_outcome'] - df['control_cf_outcome'])

In [284]:
np.mean(r_t)

In [285]:
effect = w_t*df[data['outcome_name']] + w_t*v_t*r_t

In [286]:
np.mean(effect)

In [287]:
print("MAE {}".format(np.mean(effect) - true_effect))

MAE -61.376779885166975


In [316]:
data['treatment_name']

['v0']

In [328]:
treatment = data['treatment_name'][0]
outcome = data['outcome_name']
features = data['common_causes_names'] + data['treatment_name']
delta_seq = np.linspace(0.5, 10, 10)
logreg = LogisticRegression()
logreg.fit(df[data['common_causes_names']], df[treatment])

## Compute propensity score
df['ps_1'] = logreg.predict_proba(df[data['common_causes_names']])[:,1]
df['ps_0'] = 1 - df['ps_1']

# ## Compute incremental propensity score
# df['incre_ps_1'] = (delta * df['ps_1']) / (delta * df['ps_1'] + df['ps_0'])
# df['incre_ps_0'] = 1 - df['incre_ps_1']

## Fit outcome
rf = GradientBoostingRegressor(random_state=0, n_estimators = 1000)
rf.fit(df[features], df[outcome])

## Compute counterfactual outcome with no treatment
df_pos = df.copy()
df_pos[treatment] = 1
df['treated_cf_outcome'] = rf.predict(df_pos[features])

## Compute counterfactual outcome with treatment
df_neg = df.copy()
df_neg[treatment] = 0
df['control_cf_outcome'] = rf.predict(df_neg[features])

effect = []
for delta in tqdm(delta_seq):
    df['incre_ps_1'] = (delta * df['ps_1']) / (delta * df['ps_1'] + df['ps_0'])
    df['incre_ps_0'] = 1 - df['incre_ps_1']
    
    w_t = (delta*df[treatment] + 1 - df['v0']) / (delta*df['ps_1'] + df['ps_0'])
    
    v_t = (df[treatment]*(df['ps_0']) 
                    - (1 - df[treatment])*df['ps_1']*delta) / (delta / (1-delta))

    r_t = (df['treated_cf_outcome']*df['incre_ps_1'] + df['control_cf_outcome']*df['incre_ps_0'])
    
    e = w_t*df[data['outcome_name']] + w_t*v_t*r_t
    
# Y - mu(X,A)
    residual = df[data['outcome_name'][0]] - df['predicted_y']

    w0 = [0 if df.loc[i, 'v0'] == 1 else 1/df.loc[i, 'ps_0'] for i in range(len(df))]
    w1 = [0 if df.loc[i, 'v0'] == 0 else 1/df.loc[i, 'ps_1'] for i in range(len(df))]
    w0 = np.array(w0)
    w1 = np.array(w1)

    phi_0 = w0 * residual + df['treated_cf_outcome'] 
    phi_1 = w1 * residual + df['control_cf_outcome']

    m0 = df['incre_ps_1']*phi_1 + df['incre_ps_0']*phi_0

    m1 = (delta*(df['treated_cf_outcome'] - df['control_cf_outcome'])
         *(df['v0'] - df['ps_1']))/(delta*df['ps_1'] + df['ps_0'])**2

    
    effect.append(np.mean(e))

100%|██████████| 10/10 [00:00<00:00, 269.63it/s]


In [329]:
effect

In [330]:
np.mean(effect) - true_effect